In [1]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load pre-trained model tokenizer (vocabulary)
tokenizer = AutoTokenizer.from_pretrained("camembert-base")

# Load pre-trained model (weights)
model = AutoModel.from_pretrained("camembert-base")

# Text to encode
text = "Je suis un étudiant."

# Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
input_ids = tokenizer.encode(text, add_special_tokens=True) 
tokens = tokenizer.convert_ids_to_tokens(input_ids) # Convert the ids to tokens.

# Create a tensor out of the input ids and move it to the GPU if available
input_ids = torch.tensor([input_ids])

# If you have a GPU, put everything on cuda
#input_ids = input_ids.to('cuda')
#model.to('cuda')

# This context manager makes sure that the model's internal states such as the 
# computation graph, the gradients, etc... are not saved in memory which would 
# lead to increased memory usage.
with torch.no_grad():
    last_hidden_states = model(input_ids)[0]  # Models outputs are now tuples

# Get the embeddings of the [CLS] token for each layer
word_embeddings = last_hidden_states[0]

c:\Users\Frank\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Frank\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Frank\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see ac

In [5]:
from transformers import CamembertTokenizer, CamembertModel
import torch

tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
model = CamembertModel.from_pretrained('camembert-base')

def get_word_embedding(word):
    input_ids = tokenizer.encode(word, return_tensors='pt')
    with torch.no_grad():
        outputs = model(input_ids)
    # Use the mean of the last hidden state as the word embedding
    word_embedding = outputs.last_hidden_state.mean(dim=1)
    return word_embedding

word = "maison"
embedding = get_word_embedding(word)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
word = "maison"
embedding = get_word_embedding(word)